In [3]:
import boto3
import pandas as pd
from io import StringIO


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
bucket_name = 'clustertraces'
file_key = 'borg_traces_data.csv'

# Creating a boto3 client
s3 = boto3.client('s3')

# Object rtracing
response = s3.get_object(Bucket=bucket_name, Key=file_key)

# Load CSV
csv_content = response['Body'].read().decode('utf-8')

# Dataframe declaration
df = pd.read_csv(StringIO(csv_content))


df.head()